<a href="https://colab.research.google.com/github/angulorojasmariaclaudia-coder/TRABAJO-FINAL-TECNICAS-DE-CIENCIAS-DE-DATOS/blob/main/Configuraci%C3%B3n_de_C%C3%B3digo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **¡¡IMPORTANTE!! Para enterder el DATASET**

🎯 **OBJETIVO REAL**

Clasificar qué tan mal contaminada está cada Comunidad Autónoma (CCAA)
usando solo los contaminantes por ciudades.

\

🎯 Para clasificar la contaminación de una CCAA, necesitamos **solo los contaminantes**:
* Contaminantes: NO2 y PM2.5.
* **Air Pollution Average [ug/m3]** × **Year**
* CCAA: Comunidad Autónoma


**IMPORTANTE TAMBIÉN:**
* Años considerados: 2018 a 2022.
* Contaminantes: NO2 y PM2.5.
* Población: Urban Centres (grid).


# Paquetes

In [1]:
import pandas as pd
from google.colab import files

# Conjunto de datos

In [2]:
uploaded = files.upload()
df = pd.read_csv('causas3.csv')

Saving causas3.csv to causas3 (6).csv


In [3]:
df.shape # numero de filas y columnas

(25440, 23)

In [ ]:
df.head(5) # dataset original

# Limpieza del Conjunto de Datos

### Añadir variables CCAA

Vamos a ver las ciudades que reúne nuestro conjunto de datos.

In [4]:
df["City Or Territory"].unique()

array(['Collado Villalba', 'Terrassa', 'Toledo', 'Santiago de Compostela',
       'Sevilla', 'Santa Monica', 'Santander', 'Córdoba', 'Tomares',
       'Talavera de la Reina', 'Torrejón de Ardoz', 'Coslada',
       'Tarragona', 'Torrelavega', 'Cuenca', 'Dos Hermanas',
       'Torremolinos', 'Eivissa', 'Torrevieja', 'el Masnou',
       "El Port de Sagunt / Canet d'En Berenguer", 'Valdemoro',
       'El Prat de Llobregat', 'Valencia', 'Valladolid', 'Elche/Elx',
       'Vigo', 'Elda', 'Viladecans', 'Ferrol', 'Vilanova i la Geltrú',
       'Fuengirola', 'Vitoria/Gasteiz', 'Fuenlabrada', 'Gandia', 'Zamora',
       'Gijón', 'Zaragoza', 'Girona', 'Granada', 'Granollers',
       'Guadalajara', 'Huelva', 'Igualada', 'Irun', 'Jaén',
       'Jerez de la Frontera', 'León', 'Linares',
       'Línea de la Concepción, La', 'Lleida', 'Logroño', 'Lorca', 'Lugo',
       'Madrid', 'Majadahonda', 'Málaga', 'Manresa', 'Marbella', 'Mataró',
       'Melilla', 'Mérida', 'A Coruña', 'Albacete', 'Mollet del Vall

Vemos que hay dos casillas, Santa Mónica y All Urban Centres in a Country, que no son ciudades de España. Por lo que las quitaremos de nuestro conjunto de datos.

In [5]:
df = df[df["City Or Territory"] != "All Urban Centres in a Country"]
df = df[df["City Or Territory"] != "Santa Monica"]

A continuación, vamos a añadir una columna que indique la Comunidad Autónoma según la ciudad registrada en nuestro conjunto de datos.

In [6]:
mapa_ccaa = {

    # ANDALUCÍA
    'Sevilla': 'Andalucía',
    'Córdoba': 'Andalucía',
    'Tomares': 'Andalucía',
    'Dos Hermanas': 'Andalucía',
    'Torremolinos': 'Andalucía',
    'Fuengirola': 'Andalucía',
    'Granada': 'Andalucía',
    'Huelva': 'Andalucía',
    'Jaén': 'Andalucía',
    'Jerez de la Frontera': 'Andalucía',
    'Linares': 'Andalucía',
    'Línea de la Concepción, La': 'Andalucía',
    'Marbella': 'Andalucía',
    'Málaga': 'Andalucía',
    'Puerto de Santa María, El': 'Andalucía',
    'San Fernando': 'Andalucía',
    'Sanlúcar de Barrameda': 'Andalucía',
    'Chiclana de la Frontera': 'Andalucía',
    'Cádiz': 'Andalucía',
    'Benalmádena': 'Andalucía',
    'Algeciras': 'Andalucía',
    'Almería': 'Andalucía',
    'Alcalá de Guadaíra': 'Andalucía',

    # ARAGÓN
    'Zaragoza': 'Aragón',

    # ASTURIAS
    'Gijón': 'Asturias',
    'Oviedo': 'Asturias',
    'Avilés': 'Asturias',

    # BALEARES
    'Eivissa': 'Islas Baleares',
    'Palma de Mallorca': 'Islas Baleares',

    # CANTABRIA
    'Santander': 'Cantabria',
    'Torrelavega': 'Cantabria',

    # CASTILLA-LA MANCHA
    'Toledo': 'Castilla-La Mancha',
    'Cuenca': 'Castilla-La Mancha',
    'Guadalajara': 'Castilla-La Mancha',
    'Talavera de la Reina': 'Castilla-La Mancha',
    'Albacete': 'Castilla-La Mancha',
    'Ciudad Real': 'Castilla-La Mancha',

    # CASTILLA Y LEÓN
    'Valladolid': 'Castilla y León',
    'Zamora': 'Castilla y León',
    'León': 'Castilla y León',
    'Palencia': 'Castilla y León',
    'Ponferrada': 'Castilla y León',
    'Salamanca': 'Castilla y León',
    'Burgos': 'Castilla y León',
    'Ávila': 'Castilla y León',

    # CATALUÑA
    'Terrassa': 'Cataluña',
    'Tarragona': 'Cataluña',
    'el Masnou': 'Cataluña',
    'Viladecans': 'Cataluña',
    'El Prat de Llobregat': 'Cataluña',
    'Girona': 'Cataluña',
    'Granollers': 'Cataluña',
    'Igualada': 'Cataluña',
    'Lleida': 'Cataluña',
    'Manresa': 'Cataluña',
    'Mataró': 'Cataluña',
    'Mollet del Vallès': 'Cataluña',
    'Reus': 'Cataluña',
    'Sabadell': 'Cataluña',
    'Sant Boi de Llobregat': 'Cataluña',
    'Sant Cugat del Vallès/Rubí': 'Cataluña',
    'Barcelona': 'Cataluña',
    'Castelldefels': 'Cataluña',
    'Vilanova i la Geltrú': 'Cataluña',
    'Esplugues de Llobregat': 'Cataluña',
    'Cornellà de Llobregat': 'Cataluña',
    'Cerdanyola del Vallès': 'Cataluña',
    "L'Hospitalet de Llobregat": 'Cataluña',
    'Badalona': 'Cataluña',
    'Rubí': 'Cataluña',

    # CEUTA Y MELILLA
    'Ceuta': 'Ciudad Autónoma de Ceuta',
    'Melilla': 'Ciudad Autónoma de Melilla',

    # EXTREMADURA
    'Cáceres': 'Extremadura',
    'Badajoz': 'Extremadura',
    'Mérida': 'Extremadura',

    # GALICIA
    'Santiago de Compostela': 'Galicia',
    'Ferrol': 'Galicia',
    'Vigo': 'Galicia',
    'Pontevedra': 'Galicia',
    'A Coruña': 'Galicia',
    'Lugo': 'Galicia',
    'Ourense': 'Galicia',

    # LA RIOJA
    'Logroño': 'La Rioja',

    # MADRID
    'Collado Villalba': 'Comunidad de Madrid',
    'Torrejón de Ardoz': 'Comunidad de Madrid',
    'Coslada': 'Comunidad de Madrid',
    'Valdemoro': 'Comunidad de Madrid',
    'Madrid': 'Comunidad de Madrid',
    'Majadahonda': 'Comunidad de Madrid',
    'Fuenlabrada': 'Comunidad de Madrid',
    'Parla': 'Comunidad de Madrid',
    'Pozuelo de Alarcón': 'Comunidad de Madrid',
    'Rozas de Madrid, Las': 'Comunidad de Madrid',
    'Rivas-Vaciamadrid': 'Comunidad de Madrid',
    'San Sebastián de los Reyes': 'Comunidad de Madrid',
    'Alcalá de Henares': 'Comunidad de Madrid',
    'Leganés': 'Comunidad de Madrid',
    'Alcorcón': 'Comunidad de Madrid',
    'Getafe': 'Comunidad de Madrid',
    'Alcobendas': 'Comunidad de Madrid',
    'Móstoles': 'Comunidad de Madrid',

    # MURCIA
    'Murcia': 'Región de Murcia',
    'Lorca': 'Región de Murcia',
    'Cartagena': 'Región de Murcia',

    # NAVARRA
    'Pamplona/Iruña': 'Navarra',

    # PAÍS VASCO
    'Vitoria/Gasteiz': 'País Vasco',
    'Bilbao': 'País Vasco',
    'Irun': 'País Vasco',
    'Santurtzi': 'País Vasco',
    'Barakaldo': 'País Vasco',
    'Getxo': 'País Vasco',
    'San Sebastián/Donostia': 'País Vasco',

    # COMUNIDAD VALENCIANA
    'Gandia': 'Comunidad Valenciana',
    'Torrevieja': 'Comunidad Valenciana',
    'Elche/Elx': 'Comunidad Valenciana',
    'Alicante/Alacant': 'Comunidad Valenciana',
    'Elda': 'Comunidad Valenciana',
    "El Port de Sagunt / Canet d'En Berenguer": 'Comunidad Valenciana',
    'Benidorm': 'Comunidad Valenciana',
    'Castellón de la Plana/Castelló de la Plana': 'Comunidad Valenciana',
    'Alcoi/Alcoy': 'Comunidad Valenciana',
    'Alcoy': 'Comunidad Valenciana',
    'Sagunto': 'Comunidad Valenciana',
    'Torrent': 'Comunidad Valenciana',
    'Paterna': 'Comunidad Valenciana',
    'Mislata': 'Comunidad Valenciana',
    'San Vicente del Raspeig/Sant Vicent del Raspeig': 'Comunidad Valenciana',
    'Valencia': 'Comunidad Valenciana'
}

df["CCAA"] = df["City Or Territory"].map(mapa_ccaa)


In [7]:
df["CCAA"].unique() # visualizamos los resultados

array(['Comunidad de Madrid', 'Cataluña', 'Castilla-La Mancha', 'Galicia',
       'Andalucía', 'Cantabria', 'Islas Baleares', 'Comunidad Valenciana',
       'Castilla y León', 'País Vasco', 'Asturias', 'Aragón', 'La Rioja',
       'Región de Murcia', 'Ciudad Autónoma de Melilla', 'Extremadura',
       'Navarra', 'Ciudad Autónoma de Ceuta', nan], dtype=object)

### Filtrado filas y columnas DataFrame Original

Filtramos las filas por:
* Ciudades que son de `Urban Centres (grid)`
* Los que se atribuyen a las muertes `Health Indicator = Attributable deaths (AD)`.
* Los contaminantes NO2	y PM2.5.
* Por columnas nos quedaremos con `CCAA`,`City Or Territory`, `Air Pollutant`, `Year`, `Air Pollution Average [ug/m3]` y `Air Pollution Population Weighted Average [ug/m3]`.

In [8]:
df = df[
    (df["City Boundary Specification (LAU/grid)"] == "Urban Centres (grid)") &
    (df["Health Indicator"] == "Attributable deaths (AD)") &
    (df["Air Pollutant"] != "O3")
]

Vamos a seleccionar columnas de interés y eliminar duplicados.

In [9]:
columnas = ['CCAA','City Or Territory','Air Pollutant','Year',
            'Air Pollution Average [ug/m3]',
            'Air Pollution Population Weighted Average [ug/m3]']

df = df[columnas].copy()
df = df.drop_duplicates()

### Número de ciudades y características por ciudad

Comprobamos que todas las ciudades reúnen las mismas características de estudio.

In [ ]:
pd.set_option('display.max_rows', None)
df["City Or Territory"].value_counts()

Vemos cuántas ciudades hay por CCAA.

In [ ]:
df.groupby("CCAA")["City Or Territory"].nunique().sort_values(ascending=False)

In [ ]:
df[df["City Or Territory"] == "Logroño"] # ejemplo de ciudad (todas tienen las mismas caracteríticas)

### Preparar el conjunto de datos para el algoritmo

**1.** Seleccionamos el valor de contaminación a usar `Air Pollution Average [ug/m3]`: Se crea una nueva columna llamada `valor`, que contiene el dato numérico de contaminación proveniente de Air Pollution Average [ug/m3].

Esta columna será la base para construir el dataset final.

**2.** Transformar la tabla al formato “una fila por ciudad”
Se utiliza pivot_table() para reorganizar el dataset, de modo que:

* Cada ciudad aparece en una única fila (index = "City Or Territory").
* Cada combinación de contaminante y año pasa a ser una columna.
* El contenido de esas columnas es el valor de contaminación (valor).

In [11]:
df["valor"] = df["Air Pollution Average [ug/m3]"]

df_final = df.pivot_table(
    index="City Or Territory",
    columns=["Air Pollutant", "Year"],
    values="valor"
)

df_final.columns = [
    f"{pollutant}_{year}" for pollutant, year in df_final.columns
]

df_final["CCAA"] = df_final.index.map(mapa_ccaa)

df_final = df_final.reset_index()

cols = ["CCAA", "City Or Territory"] + [
    col for col in df_final.columns if col not in ["CCAA", "City Or Territory"]
]
df_final = df_final[cols]
df_final = df_final.sort_values(by=["CCAA", "City Or Territory"]).reset_index(drop=True)

# **DATOS "LIMPIOS"**

El dataset tiene 1040 instancias y 6 variables de estudio.

In [17]:
print('Número de filas: ', df_final.shape[0])
print('Número de columnas: ', df_final.shape[1])

Número de filas:  104
Número de columnas:  12


In [13]:
df_final

,CCAA,City Or Territory,NO2_2018,NO2_2019,NO2_2020,NO2_2021,NO2_2022,PM2.5_2018,PM2.5_2019,PM2.5_2020,PM2.5_2021,PM2.5_2022
0,Andalucía,Alcalá de Guadaíra,17.9,16.0,13.8,13.5,14.1,10.2,10.2,11.2,11.1,12.7
1,Andalucía,Algeciras,19.8,19.3,16.4,14.9,15.2,12.4,11.0,10.7,9.1,10.7
2,Andalucía,Almería,18.7,15.9,12.0,10.9,11.9,15.7,18.6,13.6,11.0,15.4
3,Andalucía,Chiclana de la Frontera,11.3,11.0,8.4,8.7,9.9,10.1,10.8,9.3,7.7,8.1
4,Andalucía,Cádiz,18.8,14.8,11.2,11.7,13.9,11.1,9.7,9.5,7.8,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...
99,País Vasco,San Sebastián/Donostia,16.5,14.1,12.4,13.2,13.3,11.0,9.4,8.0,8.8,9.4
100,País Vasco,Vitoria/Gasteiz,18.1,18.4,15.1,15.0,16.7,11.1,10.2,9.0,9.0,10.0
101,Región de Murcia,Cartagena,17.1,14.7,12.3,10.4,12.8,10.4,12.2,12.4,8.9,13.3
102,Región de Murcia,Lorca,14.9,13.8,11.5,11.0,11.8,10.3,11.9,11.8,8.6,12.0
